# Calculate 3-30-300 rule per district

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ac-willeke/urban-climate/blob/main/notebooks/01_FROST_extract_climate_data.ipynb) [![github](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)](https://github.com/ac-willeke/)

**Author**: Willeke A'Campo

**Description:** This notebooks shows how to calculate the 3-30-300 rule for Kristiansand

### Map the indicators for the 3-30-300-rule

In [1]:
import os
import leafmap
import pandas as pd

In [3]:
import logging
from py_scripts.logger import setup_logging
setup_logging(default_name="nb_2023.12_map")
logger = logging.getLogger(__name__)

[INFO]: Logging configuration file found and loaded.


In [ ]:

# TODO move 
municipality = "kristiansand"
src_crs = "EPSG:25832"
reporting_dir = r"/workspaces/urban-climate/data/08_reporting/"

input_file = os.path.join(reporting_dir,f"{municipality}_district_treeVis_stat.parquet" )
# add layers to gdf for mapping
gdf_mapping = leafmap.read_parquet(
    input_file, 
    return_type='gdf', 
    src_crs= src_crs , 
    dst_crs="EPSG:4326"
    )

# Calculate the center of the study_area GeoDataFrame
center = gdf_mapping.geometry.unary_union.centroid

display(gdf_mapping[[
    "kommunenavn","grunnkretsnummer", "grunnkretsnavn", 
    "n_trees", "n_bldg", "n_res_bldg", 
    "n_trees_near_rbldg","perc_near_rbldg","labels_near_rbldg",
    "a_crown", "perc_crown", "labels_perc_crown",
    "n_res_bldg_near_gs","perc_near_gs", "labels_near_gs",
    "a_open_space", "a_private_space", "a_public_space"
    ]])

kommunenavn grunnkretsnummer             grunnkretsnavn  n_trees  n_bldg  \
0   Kristiansand         42040701           Grim - Dueknipen      298     204   
1   Kristiansand         42040702                  Grimsmyra      632     609   
2   Kristiansand         42040704     Møllevannet - Klappane      154      82   
3   Kristiansand         42040705            Enrum - Paradis      433     190   
4   Kristiansand         42040801        Kvadraturen Sørvest      440     307   
5   Kristiansand         42040802       Kvadraturen Nordvest      439     416   
6   Kristiansand         42040803         Kvadraturen Sørøst      432     375   
7   Kristiansand         42040804        Kvadraturen Nordøst      216     695   
8   Kristiansand         42040806                         Eg      366      86   
9   Kristiansand         42040807                  Sykehuset     1291      99   
10  Kristiansand         42040901                  Galgeberg      333     285   
11  Kristiansand         42040902                  Hamreheia      481     564   
12  Kristiansand         42040903                   Kuholmen      332     395   
13  Kristiansand         42040904               Valhalla Sør      128     241   
14  Kristiansand         42040905              Valhalla Midt      276     499   
15  Kristiansand         42040907  Valhalla Nord - Kongsgård      439     303   
16  Kristiansand         42040908                    Solbygg      122      98   
17  Kristiansand         42040909              Kjempegravane       47     132   
18  Kristiansand         42040910                 Tobienborg      381     302   
19  Kristiansand         42040911                 Nedre Lund      402     526   
20  Kristiansand         42040912                  Oddemarka      118     108   
21  Kristiansand         42040913                     Flaten      103     162   
22  Kristiansand         42040915     Gimlemoen - Jegersberg      580      40   
23  Kristiansand         42040918                  Gimlevang       34      42   
24  Kristiansand         42040919                      Gimle     1175      21   
25  Kristiansand         42040920               Marviksletta       48       6   
26  Kristiansand         42040921        Lund Industriområde       40       9   
27  Kristiansand         42041101            Nedre Kongsgård     1008     390   
28  Kristiansand         42041102         Kongsgard 1 - Vige      166       9   
29  Kristiansand         42041103              Bjørndalsheia      129      38   
30  Kristiansand         42041104          Vestre Bjørndalen       36      20   

    n_res_bldg  n_trees_near_rbldg  perc_near_rbldg labels_near_rbldg  \
0           87                  88               30            25-50%   
1          310                 509               81           75-100%   
2           40                  70               45            25-50%   
3           78                 141               33            25-50%   
4           72                 108               25             0-25%   
5          104                 108               25             0-25%   
6          170                 165               38            25-50%   
7          382                 163               75            50-75%   
8           32                 158               43            25-50%   
9            1                  11                1             0-25%   
10         129                 173               52            50-75%   
11         250                 415               86           75-100%   
12         160                 228               69            50-75%   
13         124                 120               94           75-100%   
14         261                 242               88           75-100%   
15         155                 328               75            50-75%   
16          50                  63               52            50-75%   
17          61                  45               96           75-100%   
18         148           

**Rule 3 | all residents should be able to see at least 3 trees from their home.**

In [ ]:
m = leafmap.Map()
m.set_center(center.x, center.y, zoom=13)
m.add_basemap("CartoDB.Positron")
m.add_data(
    gdf_mapping, column='perc_near_rbldg', scheme='Quantiles', cmap='Greens', legend_title='Number of Trees near Residential Buildings'
)
m

ImportError: mapclassify is required for this function. Install with `pip install mapclassify`.

**Rule 30: all neighbourhoods should have 30% tree canopy cover.**

In [ ]:
m = leafmap.Map()
m.set_center(center.x, center.y, zoom=13)
m.add_basemap("CartoDB.Positron")
m.add_data(
    gdf_mapping, column='perc_crown', scheme='Quantiles', cmap='Greens', legend_title='Crown Area (%)'
)
m

Map(center=[58.15207606924088, 8.004060322278963], controls=(ZoomControl(options=['position', 'zoom_in_text', …

**Rule 300:  all residents should be able to access a green area within 300 m from their home.**

In [ ]:
gdf_mapping_gs = gdf_mapping.dropna(subset=['n_res_bldg_near_gs_norm'])
m = leafmap.Map()
m.set_center(center.x, center.y, zoom=13)
m.add_basemap("CartoDB.Positron")
m.add_data(
    gdf_mapping_gs, column='perc_near_gs', scheme='Quantiles', cmap='Greens', legend_title='Number of Residential Buildings near green spaces'
)
m

Map(center=[58.15207606924088, 8.004060322278963], controls=(ZoomControl(options=['position', 'zoom_in_text', …

**TESTING OF LEAFMAP CUSTOM CATEGORICAL MAPPING**

In [ ]:

#gdf_mapping_gs = gdf_mapping.dropna(subset=['n_res_bldg_near_gs_norm'])
gdf_mapping_gs = gdf_mapping
# Define the bins
bins = pd.IntervalIndex.from_tuples([(-0.01, 25), (25, 50), (50, 75), (75, 100)])

# Create a new column 'categories' based on these bins
gdf_mapping_gs['bins'] = pd.cut(gdf_mapping_gs['n_res_bldg_near_gs_norm'], bins)
gdf_mapping_gs['bins'] = gdf_mapping_gs['bins'].astype(str)

# Define a color mapping for categories
labels = ["no residential buildings", "0-25%", "25-50%", "50-75%", "75-100%"]
color_dict = {"nan":"no residential buildings", "(-0.01, 25.0]": "0-25%", "(25.0, 50.0]": "25-50%", "(50.0, 75.0]": "50-75%", "(75.0, 100.0]": "75-100%"}

# Apply the color mapping to the 'color' column
gdf_mapping_gs['labels'] = gdf_mapping_gs['bins'].map(color_dict)

# Display the DataFrame
display(gdf_mapping_gs[[
    "kommunenavn","grunnkretsnummer", "grunnkretsnavn", 
    "n_bldg", "n_res_bldg", "n_res_bldg_near_gs", "n_res_bldg_near_gs_norm", 
    "bins", "labels" 
]])

kommunenavn grunnkretsnummer             grunnkretsnavn  n_bldg  \
0   Kristiansand         42040701           Grim - Dueknipen     204   
1   Kristiansand         42040702                  Grimsmyra     609   
2   Kristiansand         42040704     Møllevannet - Klappane      82   
3   Kristiansand         42040705            Enrum - Paradis     190   
4   Kristiansand         42040801        Kvadraturen Sørvest     307   
5   Kristiansand         42040802       Kvadraturen Nordvest     416   
6   Kristiansand         42040803         Kvadraturen Sørøst     375   
7   Kristiansand         42040804        Kvadraturen Nordøst     695   
8   Kristiansand         42040806                         Eg      86   
9   Kristiansand         42040807                  Sykehuset      99   
10  Kristiansand         42040901                  Galgeberg     285   
11  Kristiansand         42040902                  Hamreheia     564   
12  Kristiansand         42040903                   Kuholmen     395   
13  Kristiansand         42040904               Valhalla Sør     241   
14  Kristiansand         42040905              Valhalla Midt     499   
15  Kristiansand         42040907  Valhalla Nord - Kongsgård     303   
16  Kristiansand         42040908                    Solbygg      98   
17  Kristiansand         42040909              Kjempegravane     132   
18  Kristiansand         42040910                 Tobienborg     302   
19  Kristiansand         42040911                 Nedre Lund     526   
20  Kristiansand         42040912                  Oddemarka     108   
21  Kristiansand         42040913                     Flaten     162   
22  Kristiansand         42040915     Gimlemoen - Jegersberg      40   
23  Kristiansand         42040918                  Gimlevang      42   
24  Kristiansand         42040919                      Gimle      21   
25  Kristiansand         42040920               Marviksletta       6   
26  Kristiansand         42040921        Lund Industriområde       9   
27  Kristiansand         42041101            Nedre Kongsgård     390   
28  Kristiansand         42041102         Kongsgard 1 - Vige       9   
29  Kristiansand         42041103              Bjørndalsheia      38   
30  Kristiansand         42041104          Vestre Bjørndalen      20   

    n_res_bldg  n_res_bldg_near_gs  n_res_bldg_near_gs_norm           bins  \
0           87                  61                     70.0   (50.0, 75.0]   
1          310                 286                     92.0  (75.0, 100.0]   
2           40                  40                    100.0  (75.0, 100.0]   
3           78                  77                     99.0  (75.0, 100.0]   
4           72                   0                      0.0  (-0.01, 25.0]   
5          104                   0                      0.0  (-0.01, 25.0]   
6          170                  13                      8.0  (-0.01, 25.0]   
7          382                  54                     14.0  (-0.01, 25.0]   
8           32                  32                    100.0  (75.0, 100.0]   
9            1                   1                    100.0  (75.0, 100.0]   
10         129                 129                    100.0  (75.0, 100.0]   
11         250                 248                     99.0  (75.0, 100.0]   
12         160                 160                    100.0  (75.0, 100.0]   
13         124                 114                     92.0  (75.0, 100.0]   
14         261                  82                     31.0   (25.0, 50.0]   
15         155                 155                    100.0  (75.0, 100.0]   
16          50                  50                    100.0  (75.0, 100.0]   
17          61                  61                    100.0  (75.0, 100.0]   
18         148                 148                    100.0  (75.0, 100.0]   
19         187                 177                     95.0  (75.0, 100.0]   
20          31                  31                    100.0  (

In [ ]:
import leafmap

# Initialize the map and set the center
m = leafmap.Map()
m.set_center(center.x, center.y, zoom=13)

# Add the base map
m.add_basemap("CartoDB.Positron")

# Add the data to the map
m.add_data(
    data=gdf_mapping_gs,
    name="test",
    column="labels",
    colors=['#169200','#169200','#FFFFFF','#FFAA00','#FF5500'],
    labels=["no residential buildings", "0-25%", "25-50%", "50-75%", "75-100%"],
    legend_position='bottomright',
    style_kwds=dict(color="gray", LineWidth=10, weight=0.99),
    style={
        "stroke": True,
        "color": "gray",
        "weight": .5,
        "opacity": .5,
        "fill": True,               
        "fillOpacity": 0.5,           
    }
)

# Display the map
m

Map(center=[58.15207606924088, 8.004060322278963], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
color_map =  {
        "no residential buildings": (0, 136, 55),
        "0-25%": (0, 136, 55),
        "25-50%": (127, 195, 28),
        "50-75%": (127, 195, 28),
        "75-100%": (127, 195, 28),
    }


leafmap.view_vector(gdf_mapping_gs, color_column='labels', color_map=color_map, opacity=0.5)
#leafmap.Legend(title="Wetland Type", legend_dict=color_map)
#display(gdf_mapping_gs)


Map(layers=[SolidPolygonLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7fde7a81f5e0>
[
  [
 …